In [2]:
import pandas as pd
import numpy as np

file_path = "./nasa.csv"
data = pd.read_csv(file_path, comment='#')

# 데이터 확인
print("데이터 샘플:")
print(data.head())



데이터 샘플:
   rowid     pl_name  hostname pl_letter    hd_name    hip_name  \
0      1    11 Com b    11 Com         b  HD 107383   HIP 60202   
1      2    11 UMi b    11 UMi         b  HD 136726   HIP 74793   
2      3    14 And b    14 And         b  HD 221345  HIP 116076   
3      4    14 Her b    14 Her         b  HD 145675   HIP 79248   
4      5  16 Cyg B b  16 Cyg B         b  HD 186427   HIP 96901   

          tic_id                       gaia_id  sy_snum  sy_pnum  ...  \
0   TIC 72437047  Gaia DR2 3946945413106333696        2        1  ...   
1  TIC 230061010  Gaia DR2 1696798367260229376        1        1  ...   
2  TIC 333225860  Gaia DR2 1920113512486282240        1        1  ...   
3  TIC 219483057  Gaia DR2 1385293808145621504        1        2  ...   
4   TIC 27533327  Gaia DR2 2135550755683407232        3        1  ...   

   sy_kepmagerr1  sy_kepmagerr2  \
0            NaN            NaN   
1            NaN            NaN   
2            NaN            NaN   
3         

In [3]:
# 스펙트럼 타입을 숫자로 매핑하기
spectral_type_mapping = {'O': 0, 'B': -2.0, 'A': -0.3, 'F': -0.15, 'G': -0.4, 'K': -0.8, 'M': -2.0}

# st_spectype이 NaN이 아닌지 확인하고, 문자열인지 확인 후 처리
def map_spectral_type(x):
    if pd.isna(x):
        return -1  # NaN 값일 경우 -1 반환
    if isinstance(x, str) and len(x) > 0:  # 문자열인지 확인하고 비어있지 않은지 체크
        return spectral_type_mapping.get(x[0].upper(), -1)  # 첫 글자를 기준으로 매핑
    return -1  # 문자열이 아니거나 비어있는 경우 -1 반환

# 스펙트럼 타입을 숫자로 변환
data['Spectral Type Numeric'] = data['st_spectype'].apply(map_spectral_type)

# -1로 변환된 값이 있는지 확인 (잘못된 스펙트럼 타입)
invalid_spectral_types = data[data['Spectral Type Numeric'] == -1]
if not invalid_spectral_types.empty:
    print("\n잘못된 스펙트럼 타입이 있습니다. 다음 값을 확인하세요:")
    print(invalid_spectral_types[['st_spectype', 'Spectral Type Numeric']])
else:
    print("\n모든 스펙트럼 타입이 올바르게 변환되었습니다.")

# 변환된 값 확인
print("\n변환된 스펙트럼 타입 (float 형태):")
print(data[['st_spectype', 'Spectral Type Numeric']].head())


잘못된 스펙트럼 타입이 있습니다. 다음 값을 확인하세요:
     st_spectype  Spectral Type Numeric
16           NaN                   -1.0
18           NaN                   -1.0
19           NaN                   -1.0
21           NaN                   -1.0
22           NaN                   -1.0
...          ...                    ...
5687         NaN                   -1.0
5688         NaN                   -1.0
5689        T8.5                   -1.0
5702         NaN                   -1.0
5703         NaN                   -1.0

[3716 rows x 2 columns]

변환된 스펙트럼 타입 (float 형태):
  st_spectype  Spectral Type Numeric
0      G8 III                   -0.4
1      K4 III                   -0.8
2      K0 III                   -0.8
3        K0 V                   -0.8
4        G3 V                   -0.4


In [4]:

planet = "Kepler"
host = 'kepler-22'
distance = 190
spectralType = 'G5V'
VMagnitude = 12
planetDensity=15
eccentricity=0.72
au=0.812

In [5]:
spectral_type_mapping = {'O': 0, 'B': -2.0, 'A': -0.3, 'F': -0.15, 'G': -0.4, 'K': -0.8, 'M': -2.0}

# 스펙트럼 타입을 float으로 변환하는 함수 수정
def spectral_type_to_float(spectral_type):
    """
    스펙트럼 타입(예: "F5", "G2")을 Bolometric 보정값으로 변환하는 함수.
    F5 -> -0.15 (F가 3), G2 -> -0.4 (G가 4)
    """
    if isinstance(spectral_type, str) and len(spectral_type) > 0:
        # 스펙트럼 타입의 첫 글자만 사용하여 매핑 (숫자는 무시)
        base_value = spectral_type_mapping.get(spectral_type[0].upper(), np.nan)
        return base_value
    return np.nan  # 변환이 불가능한 경우 NaN 반환

# Bolometric 보정값을 계산
bolometric_correction = spectral_type_to_float(spectralType)

# 올바르게 변환되었는지 확인
print(f"Bolometric Correction for Spectral Type {spectralType}: {bolometric_correction}")



Bolometric Correction for Spectral Type G5V: -0.4


In [6]:
# PPTX 파일에 나와 있는 공식에 따라 필요한 함수 정의
def calculate_absolute_magnitude(apparent_magnitude, distance_pc):
    """
    겉보기 등급과 거리(파섹)를 이용하여 절대 등급을 계산하는 함수
    절대 등급 공식: M = m - 5 * (log10(d) - 1)
    """

    return apparent_magnitude - 5 * (np.log10(distance_pc) - 1)

def calculate_bolometric_magnitude(absolute_magnitude, bolometric_correction):
    """
    절대 등급과 Bolometric 보정값을 이용하여 절대 복사 등급을 계산하는 함수
    """
    return absolute_magnitude + bolometric_correction

def calculate_luminosity(absolute_bolometric_magnitude, sun_bolometric_magnitude=4.74, sun_luminosity=3.828*1026):
    """
    항성의 절대 복사 등급을 이용하여 항성의 광도를 계산하는 함수
    공식: L = L_sun * 10^((M_sun - M_bol) / 2.5)
    """
    return (10 ** ((absolute_bolometric_magnitude-sun_bolometric_magnitude) / (-2.5)))

def calculate_habitable_zone_boundaries(luminosity):
    """
    항성의 광도를 이용하여 생명가능지대의 내부 및 외부 반경을 계산하는 함수
    공식: r_inner = sqrt(L / 1.1), r_outer = sqrt(L / 0.53)
    """
    r_inner = np.sqrt(luminosity / 1.1)
    r_outer = np.sqrt(luminosity / 0.53)
    return r_inner, r_outer

def calculate_habitable_zone(r_inner,r_outer,au):
    if(au>r_inner and au<r_outer):
        return 1
    else:
        return 0




In [14]:
# 절대 등급 계산
absolute_magnitude = calculate_absolute_magnitude(VMagnitude, distance)
print(f"Absolute Magnitude: {absolute_magnitude:.2f}")

# 절대 복사 등급 계산
bolometric_magnitude = calculate_bolometric_magnitude(absolute_magnitude, bolometric_correction)
print(f"Bolometric Magnitude: {bolometric_magnitude:.2f}")

# 광도 계산
luminosity = calculate_luminosity(bolometric_magnitude)
print(f"Luminosity [W]: {luminosity:.2e}")

# 생명가능지대 경계 계산
r_inner, r_outer = calculate_habitable_zone_boundaries(luminosity)
print(f"Habitable Zone Inner Radius [AU]: {r_inner:.2f}")
print(f"Habitable Zone Outer Radius [AU]: {r_outer:.2f}")

result = calculate_habitable_zone(r_inner,r_outer,au)
print(result)

Absolute Magnitude: 5.61
Bolometric Magnitude: 5.21
Luminosity [W]: 6.51e-01
Habitable Zone Inner Radius [AU]: 0.77
Habitable Zone Outer Radius [AU]: 1.11
1
